In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from math import nan

# Import Language Data

In [ ]:
# import data
lang_df = pd.read_table("populations.tsv", sep="\t", header=None)

# rename columns
lang_df.rename(columns={0: "Language", 1: "Country", 2:16, 3:17, 4:18, 5:19, 6:20, 7:21, 8:22, 9:23, 10:25, 11:"drop"}, inplace=True)

# Strip whitespace from the Language and Country columns
lang_df["Language"] = lang_df["Language"].str.strip()
lang_df["Country"] = lang_df["Country"].str.strip()

# set the index to the first two columns
lang_df.set_index(["Language", "Country"], inplace=True)


lang_df

In [ ]:
# view the first few indexes
lang_df.index[:5]

In [ ]:
eval(lang_df[17][("eng", "American Samoa")])

In [ ]:
# Grab just the total number of speakers (ie the first number in the tuple)

def extract_total_speakers(cell):

    if cell == 0:
        return 0
    
    # try to check if the cell is nan
    try:
        if np.isnan(cell):
            return 0
    except:
        pass

    eval_cell = eval(cell)
    if type(eval_cell) == tuple:
        return eval_cell[0]
    else:
        return eval_cell
    

# Create a new df where each cell has just the total number of speakers
total_speakers_df = lang_df.applymap(extract_total_speakers)
total_speakers_df

# Import Population Data

In [ ]:
pop_df = pd.read_csv("World_Population_Data.csv", header=2)

In [ ]:
pop_df = pop_df.drop(columns = ["Indicator Name", "Indicator Code", "Country Code", "Unnamed: 67"])
pop_df = pop_df.set_index("Country Name")

pop_df.columns = pop_df.columns.astype(int)

In [ ]:
pop_df

# Lang/pop v time

In [ ]:
populations_northern_europe= ["Finland", "Isle of Man", "Norway", "Denmark", "Sweden", "Estonia", "Iceland", "Latvia", "United Kingdom", "Ireland", "Lithuania"]
#No Faroe Islands

In [ ]:
edition_to_year= {13:1996, 14: 2000, 15:2005, 16:2009, 17:2014, 18:2015, 19:2016, 20:2017, 21:2018, 22:2019, 23:2020, 24:2021, 25:2022}



total_speakers_df= total_speakers_df.rename(columns=edition_to_year)
total_speakers_df= total_speakers_df.stack()
total_speakers_df= total_speakers_df.unstack("Language") 

total_speakers_df

In [ ]:
ne= total_speakers_df.loc[populations_northern_europe]

In [ ]:
merged_df = total_speakers_df.merge(pop_df, on=['Country', 'Year'], how='left')
merged_df['Population'] = merged_df['Country'].map(pop_df)